In [1]:
import math
import itertools
import numpy as np

import hoomd
import gsd.hoomd

from pair import lj

In [2]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=1)

mc = hoomd.hpmc.integrate.Sphere(default_d=0.3, default_a=0.4)
mc.shape["A"] = dict(diameter=1.0)

sigma = 1.0
epsilon = 1.0

lj = lj.format(sigma=sigma, epsilon=epsilon)
patch = hoomd.hpmc.pair.user.CPPPotential(r_cut=3.0*sigma, 
                                          code=lj,
                                          param_array=[])

mc.pair_potential = patch
sim.operations.integrator = mc
sim.create_state_from_gsd(filename='./DATA/lattice.gsd')


In [3]:
logger = hoomd.logging.Logger()
logger.add(mc, quantities=['type_shapes'])

gsd_writer = hoomd.write.GSD(filename='./DATA/random.gsd',
                             trigger=hoomd.trigger.Periodic(100),
                             mode='wb',
                             filter=hoomd.filter.All(),
                             log=logger)
sim.operations.writers.append(gsd_writer)

/Users/lina492375qw1188/opt/miniconda3/envs/patchy/lib/python3.9/site-packages/hoomd/write/gsd.py:183: FutureWarning: log keyword arguments passed to <hoomd.write.gsd.GSD object at 0x103c0dc10> is deprecated since v3.9.0. Use logger instead.
  warnings.warn(


In [4]:
sim.run(1e4)